In [4]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression, LinearRegression
# from sklearn.metrics import accuracy_score, mean_squared_error
# from colorama import Fore, Style
# from tabulate import tabulate
# from prettytable import PrettyTable

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from colorama import Fore, Style
from prettytable import PrettyTable

df_existing = pd.read_excel('NBA2324.xlsx')

# checking if the 'Win' column exists before proceeding
winning_team = 'Win'  # Update with the actual column name
if winning_team in df_existing.columns:
    # creating a binary target variable indicating whether the home team wins
    df_existing['HomeWin'] = (df_existing['Home team'] == df_existing[winning_team]).astype(int)

    # specifying the categorical columns for one-hot encoding
    categorical_columns = ['Away team', 'Home team', 'Overtime', 'Arena']

    # checking if categorical columns exist in the DataFrame
    missing_columns = [col for col in categorical_columns if col not in df_existing.columns]

    if not missing_columns:
        # extracting the necessary columns for the model
        X_completed = df_existing[categorical_columns]
        y_completed = df_existing['HomeWin']

        # one-hot encode categorical columns
        X_completed_encoded = pd.get_dummies(X_completed, drop_first=True)

        # splitting the completed data into training and testing sets
        X_train_completed, _, y_train_completed, _ = train_test_split(
            X_completed_encoded, y_completed, test_size=0.2, random_state=42
        )

        # choosing a model for completed games (Logistic Regression in this example)
        model_completed = LogisticRegression()

        # training the model on the completed data
        model_completed.fit(X_train_completed, y_train_completed)
    else:
        print(f"Error: Columns {missing_columns} not found in DataFrame.")
else:
    print(f"Error: Column '{winning_team}' not found in DataFrame.")

# upcoming game data
upcoming_game_data = {
    'Away team': ['Toronto Raptors', 'Cleveland Cavaliers', 'Atlanta Hawks', 'San Antonio Spurs', 'Golden State Warriors', 'New Orleans Pelicans', 'Detroit Pistons'],
    'Home team': ['Charlotte Hornets', 'Washington Wizards', 'Boston Celtics', 'Miami Heat', 'Philadelphia 76ers', 'Los Angeles Clippers', 'Sacramento Kings'],
    'Overtime': [0, 0, 0, 0, 0, 0, 0],  # placeholder for upcoming games
    'Arena': ['Spectrum Center', 'Capital One Arena', 'TD Garden', 'Kaseya Center', 'Wells Fargo Center', 'Crypto.com Arena', 'Golden 1 Center'],
}

# creating a DataFrame for the upcoming game
df_upcoming_game = pd.DataFrame(upcoming_game_data)

# ensuring the columns in df_upcoming_game_encoded match the columns used during training
df_upcoming_game_encoded = pd.get_dummies(df_upcoming_game)
missing_columns = set(X_train_completed.columns) - set(df_upcoming_game_encoded.columns)
for column in missing_columns:
    df_upcoming_game_encoded[column] = 0

# reordering columns to match the order during training
df_upcoming_game_encoded = df_upcoming_game_encoded[X_train_completed.columns]

# making predictions using the trained model for binary outcome
predicted_probabilities = model_completed.predict_proba(df_upcoming_game_encoded)

prediction_date = 'Wednesday 7th Feb 2024'

# ANSI escape codes for color
blue_color = '\033[94m'
reset_color = '\033[0m'
green_color = '\033[92m'
red_color = '\033[91m'

# data for PrettyTable
table_new = PrettyTable()
table_new.field_names = [f"{blue_color}NBA, {prediction_date}{reset_color}", f"{blue_color}Projected Winners, v1.1{reset_color}"]
table_new.align["Projected Winners"] = "l"
table_new.horizontal_char = '-'  # Use a horizontal line as a separator

# determining the average win percentage threshold for color coding
average_win_percentage_threshold = df_existing['Win'].value_counts(normalize=True).mean() * 100

for i, team in enumerate(df_upcoming_game.itertuples(), start=1):  # Starting from index 1 to skip the header row
    try:
        team1_win_percentage = df_existing[df_existing['Home team'] == team[1]]['Win'].value_counts(normalize=True).get(team[1], 0) * 100
        team2_win_percentage = df_existing[df_existing['Home team'] == team[2]]['Win'].value_counts(normalize=True).get(team[2], 0) * 100
    except Exception as e:
        print(f"Error calculating win percentage for teams {team[1]} and {team[2]}:", str(e))
        continue

    # Determine the team with the higher win percentage as the projected winner
    if team1_win_percentage > team2_win_percentage:
        predicted_winner = team[1]
        projected_win_percentage = team1_win_percentage
        predicted_winner_color = green_color if team1_win_percentage > average_win_percentage_threshold else red_color
    else:
        predicted_winner = team[2]
        projected_win_percentage = team2_win_percentage
        predicted_winner_color = green_color if team2_win_percentage > average_win_percentage_threshold else red_color

    team_info = f"{green_color}{team[1]} ({df_existing['Win'].eq(team[1]).sum()}-{df_existing['Loss'].eq(team[1]).sum()}){reset_color} vs {green_color}{team[2]} ({df_existing['Win'].eq(team[2]).sum()}-{df_existing['Loss'].eq(team[2]).sum()}){reset_color}"

    table_new.add_row([team_info, f"{predicted_winner_color}{predicted_winner} ({projected_win_percentage:.2f}%) {reset_color}"])

print(table_new)


+--------------------------------------------------------------+--------------------------------+
|                 NBA, Wednesday 7th Feb 2024                  |    Projected Winners, v1.1     |
+--------------------------------------------------------------+--------------------------------+
|     Toronto Raptors (17-33) vs Charlotte Hornets (10-39)     |   Toronto Raptors (43.48%)     |
|   Cleveland Cavaliers (32-16) vs Washington Wizards (9-40)   | Cleveland Cavaliers (69.23%)   |
|       Atlanta Hawks (22-28) vs Boston Celtics (38-12)        |    Boston Celtics (88.46%)     |
|       San Antonio Spurs (10-40) vs Miami Heat (27-24)        |      Miami Heat (53.85%)       |
| Golden State Warriors (22-25) vs Philadelphia 76ers (30-19)  |  Philadelphia 76ers (68.00%)   |
| New Orleans Pelicans (29-21) vs Los Angeles Clippers (34-15) | Los Angeles Clippers (82.61%)  |
|      Detroit Pistons (6-43) vs Sacramento Kings (29-20)      |   Sacramento Kings (63.64%)    |
+-------------------